In [27]:
# Import required libraries
from datetime import datetime
import hashlib
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

class Blockchain:
  def __init__(self):
      self.chain = []

  def add_block(self, data):
    if not self.chain:
      previous_hash = 'NaN'
    else:
      previous_hash = self.chain[-1]['hash']


    block = {
      'data': data,
      'timestamp': str(datetime.now()),
      'previous_hash': previous_hash
    }

    converted_block = str(block).encode()
    hashed_block = hashlib.sha512(converted_block).hexdigest()
    block['hash'] = hashed_block

    self.chain.append(block)

  def is_valid(self):
    for i in range(1, len(self.chain)):
      current_block = self.chain[i]

      data = current_block["data"]
      time = current_block["timestamp"]
      previous = current_block["previous_hash"]
      test_current_block = {"data": data, "timestamp": time, "previous_hash": previous}

      previous_block = self.chain[i-1]


      current_block_string = str(test_current_block).encode()
      current_hash = hashlib.sha512(current_block_string).hexdigest()

      if current_block['hash'] != current_hash:
        return False
      if current_block['previous_hash'] != previous_block['hash']:
        return False
    return True

  def display_chain(self):
    for block in self.chain:
      print(f"Hash: {block['hash']}")
      print(f"Data: {block['data']}")
      print(f"Previous Hash: {block['previous_hash']}")
      print(f"Timestamp: {block['timestamp']}\n")

def model():
  # X = [number of nfts, price, account balance, percentage_of_balance] Y = [0=accepted, 1=fraud_flag, 2=denied]
  data = np.array([
    [32, 3000, 28000, 0, 2],
    [12, 7000, 205800, 59, 0],
    [20, 1000, 25000, 80, 1],
    [4, 15000, 38000, 0, 2],
    [2, 20000, 100000, 40, 0],
    [9, 2000, 50000, 36, 0],
    [18, 9000, 200000, 81, 1],
    [12, 300, 4235, 85, 1],
    [45, 314, 23000, 61, 0],
    [3, 25000, 100000, 75, 1],
    [43, 1000, 86000, 50, 0],
    [17, 5000, 150000, 57, 0],
    [24, 18000, 100000, 0, 2]
  ])

  X = data[:, :4]
  y = data[:, 4].astype(int)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  model = LogisticRegression(solver="liblinear", penalty='l2' ,max_iter=1000, random_state=42)
  model.fit(X_train, y_train)

  return model

def check_transactions(model, transactions):
  logits = model.predict(transactions)
  return logits

if __name__ == "__main__":
  nft_transaction_blockchain = Blockchain()

  model = model()

  transactions = np.array([
    [22, 4000, 100000, 88],  # potential fraud
    [8, 15000, 200000, 60],  # accepted
    [34, 3000, 28000, 0],  # denied
    [3, 20000, 100000, 60],  # accepted
    [12, 1000, 50000, 24],  # accepted
    [20, 5000, 120000, 83],  # potential fraud
    [40, 2000, 80000, 100],  # potential fraud
    [40, 2000, 40000, 0],  # denied
  ])

  predictions = check_transactions(model, transactions)

  for i in range(len(transactions)):
    status = ""

    if predictions[i] == 0:
      status = "Accepted"
    elif predictions[i] == 1:
      status = "Flagged for potential fraud"
    else:
      status = "Denied"

    transaction_data = {
        'buyer_id': f"buyer_{i+1}",
        'nft': f"NFT - {i+1}",
        'quantity': int(transactions[i][0]),
        'price': int(transactions[i][1]),
        'account_balance': int(transactions[i][2]),
        'percentage_of_balance': f"{int(transactions[i][3])}%",
        'status': status
    }

    if predictions[i] == 0:
      print(f"Transaction {i+1} accepted.")
      nft_transaction_blockchain.add_block(transaction_data)
    elif predictions[i] == 1:
      print(f"Transaction {i+1} flagged for potential fraud. Please contact manager.")
    elif predictions[i] == 2:
      print(f"Transaction {i+1} denied. Insufficient funds.")

  # Display the blockchain
  print("\nBlockchain Data:")
  nft_transaction_blockchain.display_chain()

  # Verify blockchain integrity
  print(f"\nBlockchain valid: {nft_transaction_blockchain.is_valid()}")

Transaction 1 flagged for potential fraud. Please contact manager.
Transaction 2 accepted.
Transaction 3 denied. Insufficient funds.
Transaction 4 accepted.
Transaction 5 accepted.
Transaction 6 flagged for potential fraud. Please contact manager.
Transaction 7 flagged for potential fraud. Please contact manager.
Transaction 8 denied. Insufficient funds.

Blockchain Data:
Hash: 14a23119f37a091f7bfb0a1b8b29592ad0af604c748e0d28d77d5baa19112ab0814c78c7583e3b3c6618c1670a809417152a3c2263453439af6bdd6321e8e29e
Data: {'buyer_id': 'buyer_2', 'nft': 'NFT - 2', 'quantity': 8, 'price': 15000, 'account_balance': 200000, 'percentage_of_balance': '60%', 'status': 'Accepted'}
Previous Hash: NaN
Timestamp: 2025-11-12 22:28:33.859235

Hash: b6ae5a7aeb403c6866d3b31ffe73cc06c80bac1391da341175f3ce8ff0e6fd1af8ee73263cb523d9d49313de34cf2660ebe9f98fee67c7f740de388976fbbf2c
Data: {'buyer_id': 'buyer_4', 'nft': 'NFT - 4', 'quantity': 3, 'price': 20000, 'account_balance': 100000, 'percentage_of_balance': '60%',